In [1]:
import PyRAMSES
import threading
import os
import shutil
import psutil
import multiprocessing
threadLock = threading.Lock()

import sys
from prettytable import PrettyTable
import importlib
importlib.reload(sys) #prepare for PrettyTable

input_file_1 = 'settings1.dat'
input_file_2 = 'dyn_B.dat'
input_file_3 = 'volt_rat_B_plus500.dat'
default_path = 'D:\\Jupyter\\VSA_short'

cpu_num = multiprocessing.cpu_count()
threads = [None] * cpu_num
results = [None] * cpu_num

class myThread (threading.Thread):   #threading  module
    def __init__(self,start_num,stop_num,index):
        threading.Thread.__init__(self)
        self.start_num = start_num
        self.stop_num = stop_num
        self.index = index
    def run(self):   #run() method                
        threadLock.acquire() #use lock object
        results[self.index] = judge_secure(self.start_num,self.stop_num,self.index)
        threadLock.release()
    
def judge_secure(start_num,stop_num,index): #function for simulating
    secure_line = 0
    fault = 0
    
    os.chdir(default_path) #reset directory to main folder after loop
    create_folder("D:\\simulate%s"%index) #create subfolder
    
    shutil.copyfile(input_file_1,'D:\\simulate%s\\%s'%(index,input_file_1))
    shutil.copyfile(input_file_2,'D:\\simulate%s\\%s'%(index,input_file_2))
    shutil.copyfile(input_file_3,'D:\\simulate%s\\%s'%(index,input_file_3))
    
    for i in range (start_num,stop_num):
        shutil.copyfile('cmd%s.txt'%i,'D:\\simulate%s\\cmd%s.txt'%(index,i))
        shutil.copyfile('short_trip_branch%s.dst'%i,'D:\\simulate%s\\short_trip_branch%s.dst'%(index,i))
        
    os.chdir("D:\\simulate%s"%index) #change directory to subfolder
    for j in range (start_num,stop_num):
        cmd = 'cmd%s.txt'%j
        case = PyRAMSES.cfg(cmd)
        case.addOut('output%s.trace'%index)
        ram = PyRAMSES.sim()
        secure_line +=1
        try: 
            ret = ram.execSim(case)
            #print(cmd + ' is secure')
        except:
            secure_line -=1
            #print(cmd + ' is not secure')  
     
    os.chdir("D:\\")
    remove_folder("D:\\simulate%s"%index) #delete subfolders
    
    fault = stop_num - start_num - secure_line
    return fault

def get_line_num(): #get the number of total test case 
    os.chdir(default_path)
    count = 0
    for row in open(input_file_2,'r').readlines():
        if row.startswith('LINE')or row.startswith('TRFO')or row.startswith('SYNC_MACH'):
            count+=1
    return count

def create_cmd(): #extract the disturbution description in dyn.dat
    result_1 = []
    result_2 = []
    result_3 = []
    index = 0
    a=0
    b=0
    c=0
    for row in open(input_file_2,'r').readlines(): #read the information in dyn.dat
        if row.startswith('LINE'):
            line= row.split(' ') #split the line into several parts
            result_1.append(line)
            with open('short_trip_branch%s.dst'%index,'w') as short_trip_line:
                short_trip_line.writelines(' 0.000 CONTINUE SOLVER BD 0.020 0.001 0. ABL\n 1.100 BREAKER BRANCH '+result_1[a][1]+' 0 0\n 5.000 CONTINUE SOLVER BD 0.020 0.001 0. ABL\n 240.000 STOP ')
                short_trip_line.flush()
            with open('cmd%s.txt'%index, 'w') as cmd_line:
                cmd_line.writelines('%s\n%s\n%s\n\ninit.trace\nshort_trip_branch%s.dst\n\ncont.trace\ndisc.trace'%(input_file_2,input_file_3,input_file_1,index))
                cmd_line.flush()
            a+=1
            index+=1
        elif row.startswith('TRFO'):
            trfo = row.split(' ')
            result_2.append(trfo)
            with open('short_trip_branch%s.dst'%index,'w') as short_trip_trfo:
                short_trip_trfo.writelines(' 0.000 CONTINUE SOLVER BD 0.020 0.001 0. ABL\n 1.100 BREAKER BRANCH '+result_2[b][1]+' 0 0\n 5.000 CONTINUE SOLVER BD 0.020 0.001 0. ABL\n 240.000 STOP ')
                short_trip_trfo.flush()
            with open('cmd%s.txt'%index, 'w') as cmd_trfo:
                cmd_trfo.writelines('%s\n%s\n%s\n\ninit.trace\nshort_trip_branch%s.dst\n\ncont.trace\ndisc.trace'%(input_file_2,input_file_3,input_file_1,index))
                cmd_trfo.flush()
            b+=1
            index+=1
        elif row.startswith('SYNC_MACH'):
            sync_mach = row.split(' ')
            result_3.append(trfo)
            with open('short_trip_branch%s.dst'%index,'w') as short_trip_sync_mach:
                short_trip_sync_mach.writelines(' 0.000 CONTINUE SOLVER BD 0.020 0.001 0. ABL\n 1.100 BREAKER SYNC_MACH '+result_3[c][1]+' 0\n 5.000 CONTINUE SOLVER BD 0.020 0.001 0. ABL\n 240.000 STOP ')
                short_trip_sync_mach.flush()
            with open('cmd%s.txt'%index, 'w') as cmd_sync_mach:
                cmd_sync_mach.writelines('%s\n%s\n%s\n\ninit.trace\nshort_trip_branch%s.dst\n\ncont.trace\ndisc.trace'%(input_file_2,input_file_3,input_file_1,index))
                cmd_sync_mach.flush()
            c+=1
            index+=1

def remove_cmd():  #remove all cmd.txt and short_trip_branch.dat in main folder
    for root, dirs, files in os.walk(default_path):
        for name in files:
            if(name.startswith("cmd")):
                os.remove(os.path.join(root, name))
            if(name.startswith("short_trip_branch")):
                os.remove(os.path.join(root, name))
            
def create_folder(path): #create subfolder
    folder = os.path.exists(path)
    if not folder:                   
        os.makedirs(path) 
    else:
        pass            

def remove_folder(path): #delete subfolder
    folder = os.path.exists(path)
    if folder:                   
        shutil.rmtree(path) 
    else:
        pass

def main():
    record = []
    line_num = get_line_num()
    num = line_num//cpu_num
    
    create_cmd()
    for k in range (0,cpu_num):
        first_num = num*k
        last_num = num*(k+1)
        if (k == cpu_num-1):
            last_num = line_num #add the remainder into last part
        threads[k] = myThread(first_num,last_num,k)
        threads[k].start()
        record.append(threads[k])
    for t in record:
        t.join() #join() method
    faults = sum(results)
    accuracy = (line_num-faults)/line_num
    if (faults == 0):
        statement = 'Secure'
    else:
        statement = 'Insecure'
    table = PrettyTable(['File number','Number of errors','Accuracy','Secure statement']) #report the result in tabular format
    table.add_row([input_file_3,faults,'percent: {:.1%}'.format(accuracy),statement])
    print(table)
    remove_cmd()
    
if __name__ == "__main__":
    main()

+------------------------+------------------+----------------+------------------+
|      File number       | Number of errors |    Accuracy    | Secure statement |
+------------------------+------------------+----------------+------------------+
| volt_rat_B_plus500.dat |        14        | percent: 86.3% |     Insecure     |
+------------------------+------------------+----------------+------------------+
